<a href="https://colab.research.google.com/github/areias/bert_covid_sentiment/blob/main/minimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install dependencies

In [ ]:
!pip install -r drive/MyDrive/ct-bert/requirements.txt

     |████████████████████████████████| 516.2 MB 3.8 kB/s 
     |████████████████████████████████| 1.1 MB 49.9 MB/s 
     |████████████████████████████████| 170 kB 76.5 MB/s 
     |████████████████████████████████| 235 kB 74.6 MB/s 
     |████████████████████████████████| 1.2 MB 60.6 MB/s 
     |████████████████████████████████| 454 kB 71.7 MB/s 
     |████████████████████████████████| 2.9 MB 66.6 MB/s 
     |████████████████████████████████| 3.0 MB 38.8 MB/s 
     |████████████████████████████████| 57 kB 1.8 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=1e599d4465251b617cc65ce8f4040a124efdb54bbaefc9de6874a538997d0db6
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estima

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.2.0


## Prepare dataset

In [ ]:
import pandas as pd 
df=pd.read_csv("/content/drive/MyDrive/train-4.csv")

In [ ]:
# id,label,text
df=df.loc[:,["tweet_id","label", "text"]]
df.columns=['id','label', 'text']

In [ ]:
type(df['label'][0])

str

In [ ]:
df['label']=df['label'].apply(lambda x: str(x))

In [ ]:
type(df['id'][0])

numpy.float64

In [ ]:
df['id']=df['id'].apply(lambda x: int(x))

In [ ]:
type(df['id'][0])


numpy.int64

In [ ]:
df['label'][0]

'0'

In [ ]:
import numpy as np
"""
    60% - train set,
    20% - dev/validation set,
    20% - test set"""

train, dev, test = np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [ ]:
type(train['label'][0])

str

In [ ]:
import os
os.makedirs("drive/MyDrive/ct-bert/data/finetune/originals/crowdbreaks")

In [ ]:
train.to_csv("drive/MyDrive/ct-bert/data/finetune/originals/crowdbreaks/train.tsv", sep="\t",index=False)
dev.to_csv("drive/MyDrive/ct-bert/data/finetune/originals/crowdbreaks/dev.tsv", sep="\t",index=False)
test.to_csv("drive/MyDrive/ct-bert/data/finetune/originals/crowdbreaks/test.tsv", sep="\t",index=False)


In [ ]:
!ls

drive  sample_data


In [ ]:
!pip install spacy==3

     |████████████████████████████████| 12.7 MB 3.7 MB/s 
     |████████████████████████████████| 628 kB 70.4 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 9.1 MB 23.1 MB/s 
     |████████████████████████████████| 451 kB 70.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
import spacy 
print(spacy.__version__)

3.0.0


In [ ]:
# had to change line 462 again in ctbert/tensorflow_models/official/nlp/data/classifier_data_lib.py
# also changed config.py line 32 to point to ct-bert model vs. 2
%%shell 
cd drive/MyDrive/ct-bert/preprocess 
python create_finetune_data.py --run_prefix test_run --finetune_datasets crowdbreaks --model_class bert_large_uncased_wwm --max_seq_length 96 --asciify_emojis --username_filler twitteruser --url_filler twitterurl --replace_multiple_usernames --replace_multiple_urls --remove_unicode_symbols

2021-11-24 08:24:14,635 [INFO ] [__main__    ]: Processing dataset crowdbreaks...
2021-11-24 08:24:14,639 [INFO ] [__main__    ]: Reading data for for type train...
2021-11-24 08:24:16,754 [INFO ] [__main__    ]: Creating preprocessed files...
2021-11-24 08:24:18,401 [INFO ] [__main__    ]: Reading data for for type dev...
2021-11-24 08:24:20,082 [INFO ] [__main__    ]: Creating preprocessed files...
2021-11-24 08:24:20,646 [INFO ] [__main__    ]: Creating tfrecords files...
2021-11-24 08:24:21,212 [INFO ] [absl        ]: Writing example 0 of 2041
2021-11-24 08:24:21,212 [INFO ] [absl        ]: *** Example ***
2021-11-24 08:24:21,212 [INFO ] [absl        ]: guid: train-0
2021-11-24 08:24:21,213 [INFO ] [absl        ]: tokens: [CLS] arizona classrooms vulnerable to me ##as ##les outbreak : the arizona department of health services looked at herd . . . twitter ##ur ##l [SEP]
2021-11-24 08:24:21,213 [INFO ] [absl        ]: input_ids: 101 5334 12463 8211 2000 2033 3022 4244 8293 1024 1996 

In [ ]:
# change run_finetune.py and set args tpu_ip require=False nad use_tpu=False
# changed line 152 and 153 and removed bucket name
# change project name defalt to ct-bert in args

%%shell
cd drive/MyDrive/ct-bert
RUN_PREFIX=testrun                                  # Name your run
#BUCKET_NAME=ct-bert                              # Fill in your buckets name here (without the gs:// prefix)
#TPU_IP=XX.XX.XXX.X                                  # Fill in your TPUs IP here
FINETUNE_DATASET=crowdbreaks                 # Your dataset name
FINETUNE_DATA=run_2021-11-24_08-04-37_868161_test_run                       # Fill in dataset run name (e.g. run_2020-05-19_14-14-53_517063_test_run)
MODEL_CLASS=covid-twitter-bert
TRAIN_BATCH_SIZE=8
EVAL_BATCH_SIZE=8
LR=2e-5
NUM_EPOCHS=1

python run_finetune.py --run_prefix $RUN_PREFIX --model_class $MODEL_CLASS --finetune_data ${FINETUNE_DATA}/${FINETUNE_DATASET} --train_batch_size $TRAIN_BATCH_SIZE --eval_batch_size $EVAL_BATCH_SIZE --num_epochs $NUM_EPOCHS --learning_rate $LR 

2021-11-24 09:53:13.660681: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-24 09:53:13.673556: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-24 09:53:13.674156: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-11-24 09:53:13.674414: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-24 09:53:13.675997: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-24 09:53:13.677259: I tensorflow/stream_executor/plat

CalledProcessError: ignored